### **Downloading the Dataset**

#### ML-100K

In [1]:
  !wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
  !unzip ml-100k.zip
  !ls

--2021-01-27 03:57:53--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.56MB/s    in 1.3s    

2021-01-27 03:57:55 (3.56 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

#### ML-1M

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2021-01-27 03:57:58--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  4.26MB/s    in 1.3s    

2021-01-27 03:57:59 (4.26 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
__notebook_source__.ipynb  ml-100k  ml-100k.zip  ml-1m	ml-1m.zip


### **Importing Libraries**

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Importing Dataset

In [4]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

### Preparing the training set and test set

In [5]:
# In this dataset folder, there are 5 different train-test set pairs for k-fold cv // xx.base is the training set and xx.test is the test set
# But we are just using one train-test set pair

training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int') #pytorch can also work with arrays so transfer data to numpy array

test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int') #pytorch can also work with arrays so transfer data to numpy array

### Getting the number of users and movies

In [6]:
# The maximum user_id or movie_id can be either in training_set or test_set so have to do max using both

nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0]))) # index0 col = 1st column = user_id column
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1]))) # index1 col = 2nd column = movid_id column

### Converting the data into an array with "users in lines" and "movies in columns"

In [7]:
# For recommendation system, we need to struture the data in a particular manner
# each movie has to correspond to each column/variable and each observation/line corresponds to each user(1st line = user1, 2nd line = user 2 ....)

# Creating list of lists = what torch expects
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1] [data[:, 0] == id_users] # all the movie IDs each user watched
        id_ratings = data[:, 2] [data[:, 0] == id_users] # all the ratings associated with those movies in the previous line
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

### Converting the data into Torch Tensors

In [8]:
# torch = multi dimensional matrix which is way more efficient than numpy arrays for most deep learning operations

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Converting the ratings into binary ratings (1 liked) or 0 (not liked)

In [9]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

### Creating Architecture of NN

In [11]:
# Stacked AutoEncoder

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__() #super allows inheritance
        
        self.fc1 = nn.Linear(nb_movies, 20) 
        #(number of features==> in this case # of movies, number of nodes/neurons in the first hidden layer==> can be experimented with various numbers)
        
        self.fc2 = nn.Linear(20, 10) # 1st hidden layer - 2nd hidden layer full connection: 20 nodes in first hidden layer, so first number in Linear() is 20
        # 2nd hidden layer number goes into the 2nd parameter of Linear()==> here choose 10 (can be tinkered/experimented)
        
        self.fc3 = nn.Linear(10, 20) # 2nd hidden layer - 3rd hidden layer full connection: 10 nodes in second hidden layer, so first number in Linear() is 10
        # 3rd hidden layer number goes into the 2nd parameter of Linear()==> here choose 20 (can be tinkered/experimented)
        
        self.fc4 = nn.Linear(20, nb_movies)
        
        self.activation = nn.Sigmoid()
    
    def forward(self, x): #encoding --- and --- decoding
        x = self.activation(self.fc1(x)) 
        x = self.activation(self.fc2(x)) 
        x = self.activation(self.fc3(x)) 
        x = self.fc4(x)
        return x
    
sae = SAE()
criterion = nn.MSELoss() #Mean Squared Error
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) #adam? #RMSprop? ==> experimentation/tinkering possible

### Training SAE

In [12]:
nb_epoch = 200

for epoch in range(1, nb_epoch + 1): #loop over epochs
    train_loss = 0
    s = 0. #keeping track of number of users who rated at least one movie
    
    for id_user in range(nb_users): #within that epoch loop, we loop over all users
        
        input = Variable(training_set[id_user]).unsqueeze(0) 
        #don't accept vector of single dimension.... so need to add some batch for new dimension
        # 0 is the index for which new dimension will be created
        
        target = input.clone() #copy our inputs
        
        if torch.sum(target.data > 0) > 0: #target.data ==> all the ratings #checking if a user has at least one movie he/she rated
            output = sae(input) #the vector of predicted ratings
            target.require_grad = False #make sure we don't calculate the gradient with respect to the target (calculate gradient only for input)
            output[target == 0] = 0 #exclude movies which don't have rating info(rating==0) for optimization
            loss = criterion(output, target) #predicted ratings, real/true ratings
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) #1e-10: making sure denominator is not 0
            loss.backward() #in which direction do we need to adjust our weights? (increase or decrease weights?)
            
            train_loss += np.sqrt(loss.data * mean_corrector) #loss data = mean error #multiply mean_corrector for adjustment
            s += 1. #number of users who rated at least one movie
            optimizer.step() #intensity/amount of change in weights
    print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(2.2468)
epoch: 2loss: tensor(2.1009)
epoch: 3loss: tensor(2.0749)
epoch: 4loss: tensor(2.0524)
epoch: 5loss: tensor(2.0395)
epoch: 6loss: tensor(2.0342)
epoch: 7loss: tensor(2.0293)
epoch: 8loss: tensor(2.0254)
epoch: 9loss: tensor(2.0200)
epoch: 10loss: tensor(2.0154)
epoch: 11loss: tensor(2.0087)
epoch: 12loss: tensor(2.0030)
epoch: 13loss: tensor(1.9946)
epoch: 14loss: tensor(1.9867)
epoch: 15loss: tensor(1.9565)
epoch: 16loss: tensor(1.9228)
epoch: 17loss: tensor(1.9077)
epoch: 18loss: tensor(1.8923)
epoch: 19loss: tensor(1.8912)
epoch: 20loss: tensor(1.8853)
epoch: 21loss: tensor(1.8781)
epoch: 22loss: tensor(1.8781)
epoch: 23loss: tensor(1.8764)
epoch: 24loss: tensor(1.8730)
epoch: 25loss: tensor(1.8712)
epoch: 26loss: tensor(1.8695)
epoch: 27loss: tensor(1.8679)
epoch: 28loss: tensor(1.8664)
epoch: 29loss: tensor(1.8661)
epoch: 30loss: tensor(1.8635)
epoch: 31loss: tensor(1.8622)
epoch: 32loss: tensor(1.8627)
epoch: 33loss: tensor(1.8602)
epoch: 34loss: tens

### Testing RBM

In [13]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(2.2516)
